In [ ]:
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

#filtrado para suavizar los datos
from scipy.signal import savgol_filter

In [ ]:
from prophet import Prophet
# from fbprophet.deagnostics import cross_validation

# Forecasting Demanda Energía (Prophet)

In [ ]:
# Importo el archivos de datos de consumo de energia en la zona este de EE.UU.
df = pd.read_csv(r'Raw_Data/medidor_1.csv')

In [ ]:
print(df.head())
print('\n')
print(df.shape[0])

In [ ]:
# Cambio de nombre en columnas y eliminamos otra
df.rename(columns={'fechahora':'Datetime', 'demanda_activa':'y[kW]'}, inplace = True)
df.drop(columns='terminal', inplace=True)

In [ ]:
#Target_values: "y[kW]"

In [ ]:
#Convierto a tipo DateTimeIndex la columna "Datetime"
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis = 0, ascending = True, inplace = True)
df.reset_index(inplace = True, drop = True)

In [ ]:
df.head()

## Limpieza de datos

### Eliminación de datos duplicados

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
df.drop_duplicates(subset = 'Datetime', keep = 'last', inplace = True)

### Tratamiento de espacios vacios para un grupo de datos continuos

In [ ]:
df_2 = df.set_index('Datetime')
df_2.drop(['2017-08-18 09:15:00'], inplace = True)

In [ ]:
print(df_2.index.min())
print(df_2.index.max())

In [ ]:
print(f'df_2.index.freq is set to: {df_2.index.freq}')

<i>
Tener un dataset con frecuencia en "None" indica 
que existen datos que perdidos (missing). <br>
Para verificar lo dicho, podemos comparar con un rango de datos
custom e ininterrumpido
</i>

In [ ]:
# Custom range
data_range = pd.date_range(start = min(df_2.index),
                          end = max(df_2.index),
                          freq = '15min') 
#freq = '15min' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
data_range

In [ ]:
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(df_2))}')

In [ ]:
#la diferencia entre ambos df indica la cantidad de valores perdidos en el df_original
print(data_range.difference(df_2.index))

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (y[kW])
df_3 = df_2.reindex(data_range)

# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
df_3['y[kW]'].interpolate(method='linear', inplace=True)

# Con la interpolación se tiene un datetime (set de hora y dias) continuo
print(f'La df.index.freq ahora es: {df_3.index.freq}, indicando que ya no tenemos valores perdidos')
print(df_3.shape[0])

In [ ]:
df_3.head()

## Filtro savgol_filter

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_3.index, y=df_3['y[kW]'],
                         mode='lines',
                         name='Datos'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
y_filtered = df_3[["y[kW]"]].apply(savgol_filter,  window_length=5, polyorder=3)
# y_filtered.head()

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_3.index,y=df_3['y[kW]'],
                         mode='lines',
                         name='No Filtrada'))
fig.add_trace(go.Scatter(x=y_filtered.index, y=y_filtered['y[kW]'],
                         mode='lines', 
                         name='Filtrada'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

### Extraemos características de la variable Tiempo

<i>
Podemos dividir la columna de Datetime en sus diferentes componentes. <br>
Esto nos permite encontrar patrones para diferentes grupos.
</i>

In [ ]:
y_filtered['dow'] = y_filtered.index.day_of_week
y_filtered['doy'] = y_filtered.index.day_of_year
y_filtered['year'] = y_filtered.index.year
y_filtered['month'] = y_filtered.index.month
y_filtered['quarter'] = y_filtered.index.quarter
y_filtered['hour'] = y_filtered.index.hour
y_filtered['weekday'] = y_filtered.index.day_name()
y_filtered['woy'] = y_filtered.index.isocalendar().week #week of year
y_filtered['dom'] = y_filtered.index.day # Day of Month
y_filtered['date'] = y_filtered.index.date 

# número de estación del año
y_filtered['season'] = y_filtered['month'].apply(lambda month_number: (month_number%12 + 3)//3) 
# el operador aritmético // solo devuelve a parte entera de la división.

## EDA

### Graficando el consumo de energía a lo largo del tiempo

In [ ]:
#Plotyle no permite acceso directo a los index del df. ?????
y_filtered['date_and_time'] = y_filtered.index 

#Plotting
fig = px.line(y_filtered, x=['date_and_time'], y='y[kW]', title=f'Demanda kW por tiempo [{min(y_filtered.year)} - {max(y_filtered.year)}]')
fig.update_traces(line=dict(width=0.3))
fig.update_layout(xaxis_title='Date & Time', yaxis_title='Demanda Energía [kW]')
fig.show()

In [ ]:
# Estudiando la gráfica se observa un comportamiento con patron en temporadas (estación del año). 

In [ ]:
#Se puede y debe comparar con la gráfica expuesta en Mr.Dims para verificar. 

### Patrones de demanda

In [ ]:
#Podemos usar nuestras funciones de fecha y hora extraídas previamente 
#para ver si surgen patrones recurrentes de los datos agregados. 
#Tomemos, por ejemplo, la demanda de energía a lo largo del día para cada día de la semana:

In [ ]:
##### Gráfica de consumo anual

#grupo de datos por años
groups = y_filtered['y[kW]'].groupby(pd.Grouper(freq='A'))

#Configuración de los ejes
fig, axs = plt.subplots(len(groups), 1, figsize=(15,15))


for ax, (name, group) in zip(axs, groups):
    
    #grafica
    ax.plot(pd.Series(group.values))

    ax.set_xlabel('Hora del año')
    ax.set_ylabel('Consumo total')
    ax.set_title(name.year)
    plt.subplots_adjust(hspace=0.5)

In [ ]:
### plot the monthly demand variability. Looking for seasonal effects

fig, axs = plt.subplots(1, 2, figsize=(30,10))

for ax, col in zip(axs, df_3.columns):
    
    groups = df_3[col].groupby(pd.Grouper(freq='M'))
    
    df = pd.DataFrame()
    
    for name, group in groups:
        df[name.month] = pd.Series(group.values)

    df.boxplot(ax=ax)
    ax.set_xlabel('Month Year')
    ax.set_ylabel('Energy Demanded MWh')
    ax.set_title(col)
    plt.subplots_adjust(hspace=1)
    
    
plt.show()

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_1 = y_filtered.groupby(['hour', 'weekday'], as_index=False).agg({'y[kW]':'median'})
# patron_1

In [ ]:
fig = px.line(patron_1, 
              x = 'hour',
              y = 'y[kW]', 
              color='weekday', 
              title='Mediana de consumo de energia por hs por día de semana ')

fig.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig.show()

In [ ]:
# Dataframe definido para graficar el consumo horario por temporada del año. Mediana de la energía. 
patron_2 = y_filtered.groupby(['hour', 'season'], as_index=False).agg({'y[kW]':'median'})
# patron_2

In [ ]:
fig_2 = px.line(patron_2, 
                x = 'hour',
                y = 'y[kW]', 
                color='season', 
                title='Mediana de consumo de energia por hs por estación')

fig_2.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig_2.show()

In [ ]:
# Durante el verano le dan duro al aire acondicionado. 

## Partición de la serie de tiempo

In [ ]:
# Los puntos que representan datos a lo largo de una serie de tiempo pueden ser interesantes 
# en cuanto sus patrones se complementes con tendencias de subida/bajada y/o estacionalidad. 
# Según la info adquirida en el EDA esto parece ser así.

In [ ]:
print(f'El primer punto de medicion fecha/hs es: {min(y_filtered.index)}')
print(f'El último punto de medicion fecha/hs es: {max(y_filtered.index)}')

In [ ]:
# Dataframe de recort
CUTOFF_DATE = pd.to_datetime('2021-03-21')
# STOP_DATE_1 = pd.to_datetime('2019-01-01')
# STOP_DATE_2 = pd.to_datetime('2022-01-01 00:00:00')

# Separo df p/ test y df p/ train
train = y_filtered.loc[(y_filtered.index < CUTOFF_DATE)].copy()  #& (y_filtered.index >= STOP_DATE_1)
test = y_filtered.loc[(y_filtered.index >= CUTOFF_DATE)].copy() #& (y_filtered.index <= STOP_DATE_2)

In [ ]:
#Se permite recortar varias fechas porque:
#1- El comportamiento es constante en el tiempo.
#2- Alivia la carga de procesamiento en la PC.
print(f'Training shape: {train.shape}\n Testing shape: {test.shape}\n')

print(f'Proporción del train-test: {((len(test)*100)/len(y_filtered)):.2f}%\n') 

print(f'Las fechas de entrenamiento son: {min(train.index)} & {max(train.index)}')
print(f'Las fechas de test son: {min(test.index)} & {max(test.index)}')

# Prophet

Es un modelo de pronóstico de series de tiempo, diseñado para manejar las características comunes
en las series de tiempo implementadas hoy en día. <br>
La idea del modelo Prophet es ser accesible y ajustable sin necesitar tener conocimientos de lo que pasa
detrás del telón respecto al funcionamiento matemático de la serie de tiempo. <br>
Tecnicamente hablando, es una serie de tiempo descompuesta en tres términos:
<i>y(t) = g(t)+s(t)+h(t)+et</i>
<ul>
<li>g(t): trend
    <blockquote> 
        Función de tendencia que modela cambios no-periodicos en los valores de la serie de tiempo.
    </blockquote>
    </li> 
<li>s(t): seasonality
    <blockquote>   
        Función que representa cambios periodicos. 
    </blockquote>
    </li> 
<li>h(t): holidays
    <blockquote>  
        Función que representa los efectos de los días de vacaciones/feriados/findes.
    </blockquote>
    </li>
<li>et: Término de error. 
    <blockquote>  
        Representa cualquier cambio idiosincracico (herencia). Se supone normalmente distribuido. 
    </blockquote>
    </li>
</ul>

Docs Oficiales (muy utiles): __[PROPHET_DOCS](https://facebook.github.io/prophet/docs/quick_start.html)__<BR>
Teoría: __[Forecasting at Scale(pdf)](https://www.kaggle.com/robinteuwens/forecasting-energy-demand/notebook)__ <br>
Practica: __[Forecasting con Prophet](https://nextjournal.com/eric-brown/forecasting-with-prophet)__ 

In [ ]:
# Formato de datos para el modelo de Prophet 
train_prophet = train[['y[kW]']].reset_index().rename(columns = {'index': 'ds', 'y[kW]': 'y'})
test_prophet = test[['y[kW]']].reset_index().rename(columns = {'index': 'ds', 'y[kW]': 'y'})
test_prophet.info()

In [ ]:
print(train_prophet.head())
print('\n')
print(test_prophet.head())

In [ ]:
f, ax = plt.subplots(figsize=(14,5))
train_prophet.plot(kind='line', x='ds', y='y', color='blue', label='Train', ax=ax)
test_prophet.plot(kind='line', x='ds', y='y', color='red', label='Test', ax=ax)
plt.title('Energía demandada: Traning and Test data')
plt.show()

## Extensión del dataframe en tiempo

In [ ]:
# from datetime import datetime, timedelta

# inicio = datetime(2022,2,1) 
# fin = datetime(2020,1,1)

# lista_fechas = [(inicio + timedelat(hour=h)) for h in range(((fin-inicio).days + 1)*24)]

# print(f'Se agregarán: {(max(lista_fechas)-min(lista_fechas))*24}')
# print(min(lista_fechas))
# print(max(lista_fechas))

In [ ]:
# fechas_extras = pd.DataFrame(lista_fechas).rename(columns = {0: 'ds'})
# fechas_extras.head()

In [ ]:
#ACÁ AGREGA TRATAMIENTO DE INTERVALO POR 15'

In [ ]:
# test_prophet_2 = pd.concat([test_prophet, fechas_extras])
# test_prophet_2.reset_index(inplace = True, drop = True)
# # De datos duplicados, solo se mantiene la medición más reciente. 
# test_prophet_2.drop_duplicates(subset = 'ds', keep = 'last', inplace = True)
# test_prophet_2.info()

## Conditional Seasonalities

__Teoría (fundamentos): ['How does Prophet work?'](https://medium.com/analytics-vidhya/how-does-prophet-work-part-2-c47a6ceac511)__

<blockquote>
In some instances the seasonality may depend on other factors, such as a weekly seasonal pattern that is different during the summer than it is during the rest of the year, or a daily seasonal pattern that is different on weekends vs. on weekdays. These types of seasonalities can be modeled using conditional seasonalities.
</blockquote>

In [ ]:
# Del EDA podemos observar que la variación diaria en estaciones es mayor en Verano e Invierno (obviamente). 
# Destripemos los patrones de los datos para tener en cuenta la interdependencia de estas variables.

In [ ]:
# Condiciones
def is_spring(ds): 
    date = pd.to_datetime(ds)    
    return (date.month >= 3) & (date.month <=5)

def is_summer(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 6) & (date.month <=8)

def is_autumn(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 9) & (date.month <=11)

# La lógica fallaba, tuve que corregir. 
def is_winter(ds): 
    date = pd.to_datetime(ds)
    return (date.month == 12) | (date.month <=2)

# A esta función la hice de una forma distinta para que ande bien.
def is_weekend(ds):     
    return ds.dayofweek in (5, 6)

In [ ]:
# agregamos al set de entrenamiento
train_prophet['is_spring'] = train_prophet['ds'].apply(is_spring)
train_prophet['is_summer'] = train_prophet['ds'].apply(is_summer)
train_prophet['is_autumn'] = train_prophet['ds'].apply(is_autumn)
train_prophet['is_winter'] = train_prophet['ds'].apply(is_winter)
train_prophet['is_weekend'] = train_prophet['ds'].apply(is_weekend)
train_prophet['is_weekday'] = ~train_prophet['ds'].apply(is_weekend) 

# train_prophet.shape

In [ ]:
# agregamos al set de testeo
test_prophet['is_spring'] = test_prophet['ds'].apply(is_spring)
test_prophet['is_summer'] = test_prophet['ds'].apply(is_summer)
test_prophet['is_autumn'] = test_prophet['ds'].apply(is_autumn)
test_prophet['is_winter'] = test_prophet['ds'].apply(is_winter)
test_prophet['is_weekend'] = test_prophet['ds'].apply(is_weekend)
test_prophet['is_weekday'] = ~test_prophet['ds'].apply(is_weekend)

In [ ]:
# test_prophet.shape
# test_prophet[test_prophet["is_weekend"]][0:100]

In [ ]:
modelo = Prophet(
    daily_seasonality = False,
    weekly_seasonality = False,
    yearly_seasonality = False
)

modelo.add_seasonality(name='yearly', period=365.25, fourier_order = 120)

modelo.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order = 13,
                        condition_name='is_spring')
modelo.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=13, 
                        condition_name='is_summer')
modelo.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=13, 
                        condition_name='is_autumn')
modelo.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=13, 
                        condition_name='is_winter')

modelo.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_spring')
modelo.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_summer')
modelo.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_autumn')
modelo.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_winter')

modelo.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_weekend')
modelo.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_weekday')
                        
# Feriados/días festivos
modelo.add_country_holidays(country_name = 'AR')

## Predicción y gráficos de tendencias

In [ ]:
# fitting el modelo
modelo.fit(train_prophet);

#parte del dataframe en el que queremos hacer la prediccion
future = test_prophet.drop(['y'], axis=1)

# Prediciendo valores
forecast = modelo.predict(future)

pd.plotting.register_matplotlib_converters()

#graficando el componente de estacionalidad encontrado
_ = modelo.plot_components(forecast)

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
fig = modelo.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), modelo, forecast)

## Definimos función MAPE: error de porcentaje absoluto medio

In [ ]:
def mape(y_true, y_pred):
    """Error de porcentaje absoluto medio"""
    
    # conversión a vectores numpy
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Porcentaje de error
    pe = (y_true - y_pred) / y_true
    
    # valor absolutos
    ape = np.abs(pe)
    
    # Cuantificación del rendimiento en un solo nº
    mape = np.mean(ape)
    
    return f'{mape*100:.2f}%'

## Graficamos los resultados obtenidos: Curva de test y de valores predecidos

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_prophet.ds, y=test_prophet.y,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for Prophet\'s predictions: {mape(test_prophet.y, forecast.yhat)}')

## Visualización de la primer y última semana (test vs previsión)

In [ ]:
# Longitud de intervalo
interval = 24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet.iloc[:interval].ds, test_prophet.iloc[:interval].y
x_pred, y_pred = forecast.iloc[:interval].ds, forecast.iloc[:interval].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet: Pronóstico de las primeras {interval} horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras {interval} horas: {mape(y_true, y_pred)}')

In [ ]:
# Longitud de intervalo
interval = -24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet.iloc[interval:].ds, test_prophet.iloc[interval:].y
x_pred, y_pred = forecast.iloc[interval:].ds, forecast.iloc[interval:].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet: Pronóstico de las últimas {abs(interval)} horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras {abs(interval)} horas: {mape(y_true, y_pred)}')

## Ajustando Hiper-Parámetros del modelo

__Time-Serie & Hyperparameter: ['Tuning'](https://www.kaggle.com/manovirat/timeseries-using-prophet-hyperparameter-tuning/notebook#HyperParameter-Tuning-using-ParameterGrid)__

<i>El siguiente bloque de código es para guardar 150 combinaciones posibles:</i><br>

```
    from sklearn.model_selection import ParameterGrid
    params_grid = {'seasonality_mode':('multiplicative','additive'),
                   'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
                   'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
                   'n_changepoints' : [100,150,200]}

    grid = ParameterGrid(params_grid)
    print(type(grid))

    cnt = 0
    for p in grid:
        cnt = cnt+1

    print('Total de posibles modelos',cnt)
```

In [ ]:
## AJUSTA LOS VALORES ANTES DE CORRER 
'''
modelo_tuneado = Prophet(growth= 'linear',   
                         n_changepoints = 200, 
                         changepoint_range=0.95, 
                         yearly_seasonality = False,
                         weekly_seasonality=False,
                         daily_seasonality = False,                                                 
                         seasonality_mode = 'additive',
                         seasonality_prior_scale=10.0,
                         changepoint_prior_scale = 0.005,                          
                         interval_width=0.8)
                         
Yearly F: 100 / Weakly F: 15 / Daily F: 7
13.52%
'''

modelo_tuneado = Prophet(growth= 'linear',   
                         n_changepoints = 200, 
                         changepoint_range=0.95, 
                         yearly_seasonality = False,
                         weekly_seasonality=False,
                         daily_seasonality = False,                                                 
                         seasonality_mode = 'additive',
                         seasonality_prior_scale=10.0,
                         changepoint_prior_scale = 0.005,
                         interval_width=0.8)

modelo_tuneado.add_seasonality(name='yearly', period=365.25, fourier_order = 100)

modelo_tuneado.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order = 15,
                        condition_name='is_spring')

modelo_tuneado.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=15,
                        condition_name='is_summer')

modelo_tuneado.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=15,
                        condition_name='is_autumn')

modelo_tuneado.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=15,
                        condition_name='is_winter')

modelo_tuneado.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=7, 
                        condition_name='is_spring')

modelo_tuneado.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=7,
                        condition_name='is_summer')

modelo_tuneado.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=7,
                        condition_name='is_autumn')

modelo_tuneado.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=7,
                        condition_name='is_winter')

modelo_tuneado.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=7,
                        condition_name='is_weekend')

modelo_tuneado.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=7,
                        condition_name='is_weekday')
                        
# Feriados/días festivos
# modelo.add_country_holidays(country_name = 'AR')

## Nueva predicción y gráficos de tendencia

In [ ]:
# fitting el modelo
modelo_tuneado.fit(train_prophet)

#parte del dataframe en el que queremos hacer la prediccion
future_two = test_prophet.drop(['y'], axis=1)

# Prediciendo valores
tunning_forecast = modelo_tuneado.predict(future_two)

pd.plotting.register_matplotlib_converters()

#graficando el componente de estacionalidad encontrado
_ = modelo.plot_components(tunning_forecast)

In [ ]:
fig = modelo_tuneado.plot(tunning_forecast)
a = add_changepoints_to_plot(fig.gca(), modelo_tuneado, tunning_forecast)

## Graficamos los nuevos resultados obtenidos: Curva de test y de valores predecidos

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_prophet.ds, y=test_prophet.y,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=tunning_forecast.ds, y=tunning_forecast.yhat,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for Prophet\'s predictions: {mape(test_prophet.y, tunning_forecast.yhat)}')

## Visualización de la primer y última semana (test vs previsión)

In [ ]:
# Longitud de intervalo
interval = 24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet.iloc[:interval].ds, test_prophet.iloc[:interval].y
x_pred, y_pred = tunning_forecast.iloc[:interval].ds, tunning_forecast.iloc[:interval].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet: Pronóstico de las primeras {interval} horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras {interval} horas: {mape(y_true, y_pred)}')

In [ ]:
# Longitud de intervalo
interval = -24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet.iloc[interval:].ds, test_prophet.iloc[interval:].y
x_pred, y_pred = tunning_forecast.iloc[interval:].ds, tunning_forecast.iloc[interval:].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet: Pronóstico de las últimas {abs(interval)} horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras {abs(interval)} horas: {mape(y_true, y_pred)}')

## Guardo los datos obtenidos en un archivo .csv

In [ ]:
# df_y = pd.DataFrame(forecast)
# df_y.rename(columns={'ds':'DateTime', 'y':'Demanda en [MW]', 'yhat': 'Demanda pronosticada en [MW]'}, inplace = True)
# df_y.set_index('DateTime')

In [ ]:
# df_y.to_csv('forecasting_prophet.csv', columns=['Demanda pronosticada en [MW]'], encoding='utf-8')

## Pasos a seguir

In [ ]:
# Probar con extensión de tiempo

In [ ]:
# Mejorar el seguimiento de la curva de pronostico con la de test para reducir el MAPE. 

In [ ]:
# Aplicar el modelo a los 4 medidores restantes